# Предобрабатываю данные (избавляюсь от дубликатов), агрегирую до года, соединяю таблицы

In [1]:
pip install -r requirements.txt

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   --- ------------------------------------ 2.4/25.2 MB 11.8 MB/s eta 0:00:02
   ------- -------------------------------- 4.7/25.2 MB 11.8 MB/s eta 0:00:02
   ---------- ----------------------------- 6.8/25.2 MB 11.3 MB/s eta 0:00:02
   -------------- ------------------------- 9.4/25.2 MB 11.3 MB/s eta 0:00:02
   ------------------ --------------------- 11.8/25.2 MB 11.5 MB/s eta 0:00:02
   ---------------------- ----------------- 14.4/25.2 MB 11.5 MB/s eta 0:00:01
   -------------------------- ------------- 16.8/25.2 MB 11.6 MB/s eta 0:00:01
   ------------------------------ --------- 19.1/25.2 MB 11.6 MB/s eta 0:00:01
   ---------------------------------- ----- 21.8/25.2 MB 11.6 MB/s eta 0:00:01
   -------------------------------------- - 24.1/25.2 MB 11.6 MB/s eta 0:00:01


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
cd ..

c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\data


c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import polars as pl

In [65]:
trade19 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2019_V202501.csv')
trade20 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2020_V202501.csv')
trade21 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2021_V202501.csv')
trade22 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2022_V202501.csv')
country_codes = pl.read_csv('./raw_data/BACI_HS17_V202501/country_codes_V202501.csv')
sql = pl.SQLContext(register_globals=True)

In [4]:
#Структура данных. t, i , j , k --- ключ.
sql.execute('''select * from trade19 limit 1;''').collect()

t,i,j,k,v,q
i64,i64,i64,i64,f64,f64
2019,4,20,420229,0.471,0.001


In [5]:
#Проверяю, что в данных нет дублей
print(sql.execute('''select i, j, k, count(*) as cnt from trade19 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade20 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade21 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade22 group by 1, 2, 3 order by 4 desc limit 1;''').collect())

shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 757 ┆ 528 ┆ 551511 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 757 ┆ 686 ┆ 580632 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 702 ┆ 196 ┆ 630491 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 604 ┆ 484 ┆ 620193 ┆ 1   │
└─────┴─────┴────────┴─────┘


In [ ]:
# Меняю типы данных на более экономные, где это возможно, и объединяю таблицы для трёх лет
sql.execute(''' 
            create table trade_tmp as
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade20
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade21
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade22
            ;
            ''')
sql.execute('''select count(*) from trade_tmp;''').collect()

len
u32
34145929


In [ ]:
# Удаляю лишние таблицы, чтобы не засоряли оперативку
del trade20
del trade21
del trade22

In [ ]:
# Джоиню данные за 2019 год, чтобы было потом на них нормировать
# Надо сделать джоин на country_codes!!!
trade = sql.execute('''select 
                tr.t
                , tr.i
                , tr.j
                , tr.category    
                , tr.v
                , tr.q             
                , t19.v as v19
                , t19.q as q19    
            from trade_tmp tr
            inner join trade19 t19
                on tr.i = t19.i
                and tr.j = t19.j
                and tr.k = t19.k
            ;''').collect()
sql.register('trade', trade)

<SQLContext [tables:6] at 0x1b055930590>

In [12]:
trade.write_parquet('preprocessed_data/trade_hs6.parquet.gzip')

In [ ]:
# Агрегирую данные о торговле с разной гранулярностью гармонизированной системы
sql.execute('''select 
                t, i, j, left(category, 4) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs4.parquet.gzip')

In [15]:
sql.execute('''select 
                t, i, j, left(category, 2) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs2.parquet.gzip')

In [16]:
sql.execute('''select 
                t, i, j
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3
            ;''').collect().write_parquet('preprocessed_data/trade_hs0.parquet.gzip')

## Агрегирую заболеваемость и смертность

In [20]:
covid_tmp = pl.read_csv('raw_data/Our World In Data/owid-covid-data.csv')
sql.register('covid_tmp', covid_tmp)
sql.execute('''select * from covid_tmp limit 2;''').collect()

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
str,str,str,str,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,i64,str,str,str,str
"""AFG""","""Asia""","""Afghanistan""","""2020-01-05""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null
"""AFG""","""Asia""","""Afghanistan""","""2020-01-06""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null


In [24]:
#Надо подумать, нет ли тут эндогенных пропусков (т.е. если у страны нулевая заболеваемость, значит ли это, что она ничего не репортит, а не репортит 0)
sql.execute('''create table covid as
    select 
            iso_code as country
            , left(date, 4)::smallint as t
            , avg(new_cases_per_million) as cases
            , avg(new_deaths_per_million) as deaths
            , avg(new_tests_per_thousand) as tests
            /*Лучше воспрользоваться данными ООН, чтобы точно использовать доковидные данные
            , avg(population_density) as population_density
            , avg(median_age) as median_age 
            , avg(aged_65_older) as aged_65_older
            , avg(aged_70_older) as aged_70_older
            , avg(cardiovasc_death_rate) as cardiovasc_death_rate
            , avg(handwashing_facilities) as handwashing_facilities
            , avg(life_expectancy) as life_expectancy */
    from covid_tmp
    where date <= '2022-12-31'
    group by iso_code, left(date, 4)        
;''')
sql.execute('''select * from covid limit 2;''').collect()

country,t,cases,deaths,tests
str,i16,f64,f64,str
"""ALB""",2021,147.121425,1.989123,null
"""MDV""",2022,474.714493,0.261452,null


## Агрегирую коронавирусные ограничения

In [27]:
restrictions_tmp = pl.read_csv('raw_data/OxCGRT/OxCGRT_compact_national_v1.csv')
sql.register('restrictions_tmp', restrictions_tmp)
sql.execute('''select * from restrictions_tmp 
            -- where `E3_Fiscal measures` is not null and `E3_Fiscal measures` > 0 
            limit 2;''').collect()

CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,C3M_Cancel public events,C3M_Flag,C4M_Restrictions on gatherings,C4M_Flag,C5M_Close public transport,C5M_Flag,C6M_Stay at home requirements,C6M_Flag,C7M_Restrictions on internal movement,C7M_Flag,C8EV_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6M_Facial Coverings,H6M_Flag,H7_Vaccination policy,H7_Flag,H8M_Protection of elderly people,H8M_Flag,V1_Vaccine Prioritisation (summary),V2A_Vaccine Availability (summary),V2B_Vaccine age eligibility/availability age floor (general population summary),V2C_Vaccine age eligibility/availability age floor (at risk summary),V2D_Medically/ clinically vulnerable (Non-elderly),V2E_Education,V2F_Frontline workers (non healthcare),V2G_Frontline workers (healthcare),V3_Vaccine Financial Support (summary),V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
str,str,str,str,str,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,str,f64,i64,f64,i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,str,f64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,str,i64,i64,str,f64,f64,f64,f64,f64
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200101,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200102,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0


In [29]:
sql.execute('''create table restrictions as
    select CountryCode as country
            , left(Date::varchar, 4)::smallint as t
            , avg(`C8EV_International travel controls`)::float4 as C8EV_International_travel_controls
            , avg(`E1_Income support`)::float4 as E1_Income_support
            , avg(`E2_Debt/contract relief`)::float4 as E2_Debt_contract_relief
            , avg(`E3_Fiscal measures`)::float4 as E3_Fiscal_measures
            , avg(`E4_International support`)::float4 as E4_International_support
            , avg(`H6M_Facial Coverings`)::float4 as H6M_Facial_Coverings
            , avg(`StringencyIndex_Average`)::float4 as StringencyIndex_Average
            , avg(`GovernmentResponseIndex_Average`)::float4 as GovernmentResponseIndex_Average
            , avg(`ContainmentHealthIndex_Average`)::float4 as ContainmentHealthIndex_Average
            , avg(`EconomicSupportIndex`)::float4 as EconomicSupportIndex
    from restrictions_tmp
    group by CountryCode, left(Date::varchar, 4)::smallint           
;''')
sql.execute('''select * from restrictions limit 10;''').collect()

country,t,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H6M_Facial_Coverings,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
str,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""ERI""",2020,2.704918,0.0,0.770492,1490.318237,0.0,0.63388,68.366997,48.218414,52.353497,19.262295
"""UGA""",2020,2.311476,0.0,1.079235,1181480.5,0.0,2.379781,62.574646,51.545574,55.05418,26.980875
"""EST""",2021,1.049315,0.657534,0.70137,25249.820312,0.0,1.884932,38.122658,43.777233,45.461533,31.986301
"""RWA""",2021,1.761644,0.665753,1.331507,0.0,0.0,2.942466,60.956573,61.267204,64.076408,41.609589
"""MYS""",2021,2.789041,1.0,2.0,2.72049984e8,0.0,3.0,70.620682,73.066383,72.82019,74.794518
"""ARE""",2022,1.109589,0.0,0.0,null,null,2.169863,31.200302,40.630329,46.437973,0.0
"""SSD""",2021,1.652055,0.326027,0.0,0.0,0.0,2.364383,45.307178,41.671341,46.461479,8.150685
"""AGO""",2022,1.441096,0.731507,0.0,null,null,2.964384,37.656849,41.23518,44.512985,18.287672
"""IDN""",2022,1.265753,1.528767,1.528767,null,null,2.369863,39.671341,52.541042,50.16452,69.178085


## Агрегирую инструментальные переменные

In [30]:
temperature_tmp = pl.read_csv('raw_data/Our World In Data/average-monthly-surface-temperature.csv')
sql.register('temperature_tmp', temperature_tmp)
sql.execute('''select * from temperature_tmp limit 2;''').collect()

Entity,Code,year,Day,Average surface temperature,Average surface temperature_duplicated_0
str,str,i64,str,f64,f64
"""Afghanistan""","""AFG""",1940,"""1940-01-15""",-2.032494,11.327695
"""Afghanistan""","""AFG""",1940,"""1940-02-15""",-0.733503,11.327695


In [34]:
sql.execute('''create table temperature as
    select 
            Code as country
            , avg(`Average surface temperature`) as average_surface_temperature
    from temperature_tmp
    where year >= 2000 and year <= 2019
    group by Code               
;''')
sql.execute('''select * from temperature limit 2;''').collect()

country,average_surface_temperature
str,f64
"""HMD""",1.65656
"""TUN""",20.447676


In [26]:
median_age_tmp = pl.read_excel('raw_data/United Nations/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx')
sql.register('median_age_tmp', median_age_tmp)
sql.execute('''select * from median_age_tmp where `ISO3 Alpha-code` <> '' limit 2;''').collect()

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 6, falling back to string


Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)","Median Age, as of 1 July (years)","Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",Population Change (thousands),Population Growth Rate (percentage),Population Annual Doubling Time (years),Births (thousands),Births by women aged 15 to 19 (thousands),"Crude Birth Rate (births per 1,000 population)",Total Fertility Rate (live births per woman),Net Reproduction Rate (surviving daughters per woman),Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),Total Deaths (thousands),Male Deaths (thousands),Female Deaths (thousands),"Crude Death Rate (deaths per 1,000 population)","Life Expectancy at Birth, both sexes (years)",Male Life Expectancy at Birth (years),Female Life Expectancy at Birth (years),"Life Expectancy at Age 15, both sexes (years)",Male Life Expectancy at Age 15 (years),Female Life Expectancy at Age 15 (years),"Life Expectancy at Age 65, both sexes (years)",Male Life Expectancy at Age 65 (years),Female Life Expectancy at Age 65 (years),"Life Expectancy at Age 80, both sexes (years)",Male Life Expectancy at Age 80 (years),Female Life Expectancy at Age 80 (years),"Infant Deaths, under age 1 (thousands)","Infant Mortality Rate (infant deaths per 1,000 live births)",Live Births Surviving to Age 1 (thousands),"Under-Five Deaths, under age 5 (thousands)","Under-Five Mortality (deaths under age 5 per 1,000 live births)","Mortality before Age 40, both sexes (deaths under age 40 per 1,000 live births)","Male Mortality before Age 40 (deaths under age 40 per 1,000 male live births)","Female Mortality before Age 40 (deaths under age 40 per 1,000 female live births)","Mortality before Age 60, both sexes (deaths under age 60 per 1,000 live births)","Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
i64,str,str,str,i64,str,str,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2595,"""Estimates""","""Burundi""","""""",108,"""BDI""","""BI""",108,"""Country/Area""",910,1950,"""2229.322""","""2254.938""","""1080.184""","""1174.755""","""86.895""","""91.95""","""18.305""","""64.551""","""28.627""","""51.232""","""2.272""","""30.508""","""117.211""","""11.582""","""51.98""","""6.923""","""2.072""","""30.674""","""102.5""","""52.66""","""26.76""","""25.899""","""23.353""","""40.938""","""39.571""","""42.245""","""44.916""","""43.636""","""46.095""","""11.048""","""10.594""","""11.423""","""4.728""","""4.454""","""4.937""","""17.777""","""162.866""","""104.421""","""28.358""","""276.13""","""457.62""","""471.554""","""444.227""","""614.352""","""639.661""","""590.518""","""290.807""","""309.954""","""273.27""","

In [32]:
sql.execute('''create table median_age_table as
    select distinct on(country)
            `ISO3 Alpha-code` as country
            , Year as year
            , `Median Age, as of 1 July (years)`::float4 as median_age
    from median_age_tmp
    where `ISO3 Alpha-code` <> '' 
            and Year <= 2019       
    order by year desc        
;''')
sql.execute('''select * from median_age_table order by 3 desc limit 10;''').collect()

country,year,median_age
str,i64,f32
"""VAT""",2019,60.292
"""MCO""",2019,54.641998
"""SHN""",2019,48.547001
"""JPN""",2019,47.262001
"""MTQ""",2019,47.029999
"""ITA""",2019,45.998001
"""SMR""",2019,45.332001
"""DEU""",2019,44.903999
"""IMN""",2019,44.831001


In [36]:
SARS_table = pl.read_excel('raw_data/WHO/SARS.xlsx')
sql.register('SARS_table', SARS_table)
sql.execute('''select * from SARS_table limit 2;''').collect()

Areas,iso3,Female,Male,Total,Median age (range),Number of deaths^a,Case fatality ratio (%),Number of imported cases (%),Number of HCW affected (%),Date onset first probable case,Date onset last probable case
str,str,i64,i64,i64,str,i64,i64,str,str,str,str
"""Australia""","""AUT""",4,2,6,"""15 (1-45)""",0,0,"""6 (100)""","""0 (0)""","""26-Feb-03""","""1-Apr-03"""
"""Canada""","""CAN""",151,100,251,"""49 (1-98)""",43,17,"""5 (2)""","""109 (43)""","""23-Feb-03""","""12-Jun-03"""


In [40]:
density_and_mortality_tmp = pl.read_csv('raw_data/World Bank/pop_dens_and_mortality.csv')
sql.register('density_and_mortality_tmp', density_and_mortality_tmp)
sql.execute('''select * from density_and_mortality_tmp limit 2;''').collect()

Time,Time Code,Country Name,Country Code,Population density (people per sq. km of land area) [EN.POP.DNST],"Population, female (% of total population) [SP.POP.TOTL.FE.ZS]","Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]","Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]","Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]","Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]","Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]","Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]","Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]","Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]",Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS],Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]
i64,str,str,str,f64,f64,i64,str,str,str,str,f64,f64,f64,f64,f64
1990,"""YR1990""","""Afghanistan""","""AFG""",18.468424,50.096823,null,null,null,null,null,382.051,510.052,122.5,60.735996,12.86206
1990,"""YR1990""","""Albania""","""ALB""",119.946788,49.765495,null,null,null,null,null,66.103,127.21,35.2,20.653647,null


In [60]:
sql.execute('''create table density_and_mortality as
    select distinct on(country)
            `Country Code` as country
            , Time as year
            , `Population density (people per sq. km of land area) [EN.POP.DNST]` population_density
            , `Population, female (% of total population) [SP.POP.TOTL.FE.ZS]` female_ratio
            , `Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]`::float4 maternal_mortality
            , `Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]`::float4 cvd_mortality
            , `Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]`::float4 pollution_mortality
            , `Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]`::float4 poisoning_mortality 
            , `Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]`::float4 hygiene_mortality
            , `Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]`::float4 female_mortality
            , `Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]`::float4 male_mortality
            , `Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]`::float4 infant_mortality
            , `Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS]`::float4 largest_city_share
            , `Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]`::float4 urbanization 
    from density_and_mortality_tmp
    where Time <= 2019        
    order by country, year desc
;''')
sql.execute('''select * from density_and_mortality order by female_ratio limit 10;''').collect()

country,year,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization
str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""QAT""",2019,229.648129,26.710807,null,10.7,90.800003,0.3,0.4,37.437,46.422001,5.3,27.357042,null
"""ARE""",2019,133.00176,35.48185,null,18.5,70.0,0.4,0.8,44.261002,68.244003,5.0,34.558586,62.610657
"""OMN""",2019,14.834381,36.346578,null,21.5,104.0,0.9,1.6,53.222,76.462997,9.3,38.278755,32.706516
"""BHR""",2019,1902.251282,37.517391,null,16.1,68.099998,0.3,0.8,48.797001,58.199001,6.0,45.268242,null
"""KWT""",2019,249.288215,38.318484,null,11.9,45.200001,0.4,0.8,31.812,59.479,7.5,68.713997,68.713997
"""SAU""",2019,13.985179,38.831635,null,20.9,91.099998,0.8,1.9,81.362999,100.896004,6.2,27.977003,54.159729
"""MDV""",2019,1625.77,38.838087,null,11.6,32.0,0.0,2.3,41.924999,56.172001,6.0,null,null
"""SYC""",2019,212.228261,45.395279,null,21.1,43.099998,0.5,8.4,79.568001,188.007996,12.8,null,null
"""PLW""",2019,38.691304,45.982695,null,null,null,null,null,null,null,22.9,null,null


## Переменные, которые не входят в первый приоритет

In [72]:
household_size_tmp = pl.read_excel('raw_data/United Nations/undesa_pd_2022_hh-size-composition.xlsx')
sql.register('household_size_tmp', household_size_tmp)
sql.execute('''select * from household_size_tmp limit 2;''').collect()

Could not determine dtype for column 9, falling back to string
Could not determine dtype for column 10, falling back to string


Country or area,ISO Code,Data source category,Reference date (dd/mm/yyyy),Average household size (number of members),1 member,2-3 members,4-5 members,6 or more members
str,i64,str,str,f64,f64,f64,f64,f64
"""Afghanistan""",4,"""MICS""","""2010-01-07 00:00:00""",7.75,0.15,7.41,18.59,73.85
"""Afghanistan""",4,"""DHS""","""19/10/2015""",8.04,0.19,6.45,16.84,76.52


In [73]:
sql.execute('''create table household_size as
            select distinct on(country)
                cc.country_iso3 as country
                , case when `Reference date (dd/mm/yyyy)` like '__/__/____' then right(`Reference date (dd/mm/yyyy)`, 4)
                    else left(`Reference date (dd/mm/yyyy)`, 4) end ::int as year 
                , `Average household size (number of members)`::float4 avg_hh_size
                , `1 member`::float4 as share_1_members
                , `2-3 members`::float4 as share_2_3_members
                , `4-5 members`::float4 as share_6plus_members
            from household_size_tmp hh
            inner join country_codes cc
                on hh.`ISO Code` = cc.country_code
            order by 1, 2 desc
;''')
sql.execute('''select * from household_size order by 3 desc limit 10;''').collect()

country,year,avg_hh_size,share_1_members,share_2_3_members,share_6plus_members
str,i32,f32,f32,f32,f32
"""BGR""",2011,null,30.790001,48.59,17.799999
"""BRN""",2011,null,7.22,19.82,28.98
"""BWA""",2011,null,27.76,32.049999,20.83
"""CHN""",2010,null,14.53,51.23,27.6
"""COK""",2011,null,17.93,38.860001,26.83
"""CYM""",2010,null,32.57,45.709999,17.719999
"""IRN""",2016,null,8.48,49.130001,37.860001
"""KNA""",2011,null,null,null,null
"""NCL""",2004,null,17.450001,38.669998,28.23


In [49]:
sql.execute('''select `Data source category`, count(*) from household_size_tmp group by 1;''').collect()

Data source category,len
str,u32
"""MICS""",203
"""LFS""",47
"""DYB""",207
"""IPUMS""",313
"""DHS""",289
